## Overview

This code calculates the inputs parameter by assimilating to flux tower data in West Phoenix. The results are discussed in section 3.3 of the paper. 

## Instructions

This file should be run in Google Colab.

A total of 50 trials are run. Each optimisation trial uses a different set of initial parameter values. One active session in Google Colab uses 2 CPUs, allowing a maximum of 10 trials to be run simultaneously with 5 active sessions. 

**To run the file:**
1. Upload and open the file in Google Colab.
2. Click **Files** in Google Colab and upload *US-WestPhoenix_era5_corrected_v1.nc* and *591_fluxtower_data_a16dafefa6836b419580f353b76bc2a0.csv* into the file storage.
3. Run the code.
4. To open another active session, save a copy of this file and open it in another tab. Adjust the trial numbers at the bottom of the code to generate the next 10 sets of initial parameter values. 

In [ ]:
#-- Import general libraries
import numpy as np
import pandas as pd
import math
import torch
import torch.nn as nn
import xarray as xr

torch.manual_seed(0)
random_no = torch.rand(50, 5)

In [2]:
# Observations
# Time in UTC

start = 0 #time to start optimisation
Nt = 150 #total time for optimisation

no_periods = start+Nt+1
daterange_UTC = pd.date_range("2012-05-01 07:00:00", freq="1h", periods=no_periods) #local standard time = -7 UTC
daterange_local = pd.date_range("2012-05-01 00:00:00", freq="1h", periods=no_periods)
ds_era5_corrected = xr.open_dataset('US-WestPhoenix_era5_corrected_v1.nc')
data_corrected = ds_era5_corrected.sel(time=daterange_UTC)
df = pd.read_csv('591_fluxtower_data_a16dafefa6836b419580f353b76bc2a0.csv')
df2 = df.loc[df["TIMESTAMP"].between(str(daterange_local[0]), str(daterange_local[-1]))]

# Thermal & radiation properties
epsilon = 0.95
sigma = 5.67e-8
mid_day_alpha = 0.172

# Forcing data
K_down_array = data_corrected['SWdown']
L_down_array = data_corrected['LWdown']
Ta_array = data_corrected['Tair']

# Target temperatures & air temperature
target_T2 = np.array(df2['Temp_C_Avg'][0::2]+273.15)[start:]
target_T5 = np.array(df2['Temp_C_2_Avg'][0::2]+273.15)[start:]
target_T15 = np.array(df2['Temp_C_3_Avg'][0::2]+273.15)[start:]

# Convert to tensors
K_down_array = torch.tensor(np.array(K_down_array),dtype=torch.float64)
L_down_array = torch.tensor(np.array(L_down_array),dtype=torch.float64)
Ta_array = torch.tensor(np.array(Ta_array),dtype=torch.float64)

target_T2 = torch.tensor(np.array(target_T2),dtype=torch.float64)
target_T5 = torch.tensor(np.array(target_T5),dtype=torch.float64)
target_T15 = torch.tensor(np.array(target_T15),dtype=torch.float64)
target = torch.zeros(2,Nt+1, dtype=torch.float64)
target[0,:] = target_T5
target[1,:] = target_T15

In [ ]:
import multiprocessing
import concurrent.futures


# Define the function to be executed by each process
def worker_function(x):
    mse_loss = nn.MSELoss()

    # Specify size of the domain
    Nx = 101
    xa = 0
    xb = 1

    # Create a meshgrid
    x_array = np.linspace(xa, xb, Nx) # np array
    dx = (xb-xa)/(Nx-1)
    depth_measurement = 0.05
    depth_i = int(np.floor(depth_measurement/dx))
    depth_measurement = 0.15
    depth_i_2 = int(np.floor(depth_measurement/dx))

    # Time setup
    hour = 3600
    dt = hour;


    #Min & Max values of parameters
    h0 = 5
    h1 = 20
    beta0 = 0.5
    beta1 = 5
    C0 = 2e6
    C1 = 2.5e6
    Tb0 = 290
    Tb1 = 303
    Lambda0 = 0.47
    Lambda1 = 0.8

    class HeatEqConv(nn.Module):
        def __init__(self, Nx, filter_weight): #filter_weight
            super(HeatEqConv, self).__init__()

            # Specify the size of the input (batch_size, channels, width)
            input_size = (1, 1, Nx)  # Adjust the size based on your requirements
            batch_size, in_channels, width = input_size

            # Specify the size of the filter/kernel
            kernel_size = filter_weight.shape[2]

            # Create a Conv1d layer with the specified weight, input size, and padding
            self.conv_layer = nn.Conv1d(in_channels, out_channels=1,kernel_size=kernel_size, padding='valid', bias=False)
            self.conv_layer.weight.data = filter_weight

        def forward(self, previous):

            future = self.conv_layer(previous) # previous(1,1,Nx); future(1,1,Nx-2)
            return future

    # Convert NumPy arrays to PyTorch tensors with float type
    x_tensor = torch.tensor(np.array(x_array), dtype=torch.float64)

    # filter corresponding to the second order central difference of the second derivative
    filter = torch.tensor([1, 0., 1], dtype=torch.float64)

    # resize filter for PyTorch
    # filter_weight(num_kernels/output channels, kernel_height, kernel_width)
    filter_weight = filter.view(1, 1, filter.shape[0])

    # Create instance of convolution and use it as a function to apply convolution
    mymodel_time_march = HeatEqConv(Nx,filter_weight)


    def forward(h_beta,Lambda,C,Tb,mymodel_time_march,Nt,start):

        h_beta.retain_grad()
        Tb.retain_grad()
        Lambda.retain_grad()
        C.retain_grad()

        T_array = torch.zeros(2,Nt+1,dtype=torch.float64)
        #Define initial condition
        T_array[0,0] = Tb
        T_array[1,0] = Tb
        Tn1 = Tb*torch.ones(Nx,dtype=torch.float64)
        tol = 1e-4

        #Solve for temperature at measurement depth
        for t in range(start,Nt+start):
            Tn1 = HeatEqSolver(mymodel_time_march,Tn1,tol,K_down_array[t+1],L_down_array[t+1],Ta_array[t+1],h_beta,Lambda,C,Tb)
            T_array[0,t-start+1] = Tn1[depth_i]
            T_array[1,t-start+1] = Tn1[depth_i_2]

        return T_array,Tn1

    def HeatEqSolver(mymodel_time_march,Tn,tol,K_down,L_down,Ta,h_beta,Lambda,C,Tb):
        error = 9e9
        # first guess
        Tn1_k = Tn
        # recalculate r
        r = Lambda/C*dt/(dx**2)
        while(error>tol):
            # Input kth approximation of Tn+1 into NN -> transform into 3D tensor
            Tn1_k_tensor= Tn1_k.view(1, 1, Nx)
            # Apply convolution to obtain (k+1)th approximation of Tn+1 for interior nodes
            Tn1_k1_tensor = 1/(1+2*r)*(r*mymodel_time_march(Tn1_k_tensor) + Tn[1:-1])
            # Calculate new Tn+1 at boundary based on Tn and kth approximation of Tn+1
            Tn1_k1_0 = 1/(1+2*r*(1+dx/Lambda*(h_beta+sigma*epsilon*Tn1_k[0]**3)))*(Tn[0]+2*r*(Tn1_k[1]+dx/Lambda*(K_down*(1-mid_day_alpha) + L_down + h_beta*Ta)))
            # Append BCs
            Tn1_k1_tensor  = torch.cat((Tn1_k1_0, Tn1_k1_tensor[0,0,:], Tb),0)
            # Calculate error
            error = torch.max(torch.abs(Tn1_k1_tensor-Tn1_k))
            # Continue from (k+1)th approximation
            Tn1_k = Tn1_k1_tensor


        return Tn1_k1_tensor

    print(r'Trial no: {} '.format(x))

    #Random initialisation of parameters
    Tb = torch.tensor([Tb0+(Tb1-Tb0)*random_no[x,4]], dtype=torch.float64,requires_grad=True)
    h = h0+(h1-h0)*random_no[x,0]
    beta = beta0+(beta1-beta0)*random_no[x,1]
    C = torch.tensor([C0+(C1-C0)*random_no[x,2]], dtype=torch.float64,requires_grad=True)
    Lambda = torch.tensor([Lambda0+(Lambda1-Lambda0)*random_no[x,3]], dtype=torch.float64,requires_grad=True)
    h_beta = torch.tensor([h*(1+1/beta)], dtype=torch.float64,requires_grad=True)

    losses = [9e9]

    its_max = 150

    optimizer = torch.optim.Adam([{'params':[h_beta,Tb]},{'params':[Lambda], 'lr':0.2},{'params':[C], 'lr':1e5}],lr=1)

    data_values = np.zeros([2,4])
    rel_change_loss = math.inf
    delta_pp = [9e9,9e9,9e9,9e9]

    n = 0

    while max(delta_pp) > 0.1 or rel_change_loss > 0.01:
      if n == 0:
        data_values[0,:] = [h_beta.item(),Lambda.item(),C.item(),Tb.item()]

      optimizer.zero_grad()
      [output,Tn1] = forward(h_beta,Lambda,C,Tb,mymodel_time_march,Nt,start)
      loss = mse_loss(output,target)
      hbeta_prev = h_beta.item()
      Lambda_prev = Lambda.item()
      C_prev = C.item()
      Tb_prev = Tb.item()

      # Backward pass to calculate gradients
      loss.backward(retain_graph=True)

      rel_change_loss = abs(losses[-1]-loss.item())/losses[-1]*100

      losses.append(loss.item())

      # optimize parameters
      optimizer.step()

      #calculate relative % change in parameters
      delta_pp = [abs(hbeta_prev-h_beta.item())/hbeta_prev*100,abs(Lambda_prev-Lambda.item())/Lambda_prev*100,abs(C_prev-C.item())/C_prev*100,abs(Tb_prev-Tb.item())/Tb_prev*100]

      if n%5 == 0:
        print(r'Trial no.: {}, iterations: {}, loss: {:.5f}, rel change: {:.5f} % \n'.format(x,n,loss.item(),rel_change_loss))
        print(r'Trial no.: {}, h_beta:{:.5f}, Lambda:{:.5f}, C:{:.5f}, Tb:{:.5f} \n'.format(x,h_beta.item(),Lambda.item(),C.item(),Tb.item()))
        print(r'Trial no.: {}, h_beta_delta:{:.5f}%, Lambda_delta:{:.5f}%, C_delta:{:.5f}% \n'.format(x,delta_pp[0],delta_pp[1],delta_pp[2],delta_pp[3]))
        print('\n')

      n = n+1
      if n > its_max:
        break


    data_values[1,:] = [h_beta.item(),Lambda.item(),C.item(),Tb.item()]

    return(data_values)

if __name__ == '__main__':

    with concurrent.futures.ProcessPoolExecutor(max_workers=2) as executor:
      inputs = list(range(10)) #first 10 trials
      resultsi = executor.map(worker_function, inputs)



In [ ]:
results_temp = list(resultsi)

In [ ]:
with open('final_parameter_values_WP_1.txt','w') as outfile:
  for trial_no in range(10):
    outfile.write('#Trial no.{}\n'.format([trial_no]))
    np.savetxt(outfile,results_temp[trial_no],fmt='%-7.5e')

from google.colab import files
files.download('final_parameter_values_WP_1.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>